# Esqueleto: Freesound Audio Tagging

---
## 1. Configuración del Entorno

### 1.1 Instalación de Librerías y Montaje del Drive

In [ ]:
# Para el registro de experimentos
!pip install -q comet_ml
!pip install librosa

In [2]:
# Montar Google Drive para acceder a los datos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1.2 Imports y Configuración de Semillas

In [3]:
import os
import librosa
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Audio

import comet_ml
from google.colab import userdata

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print(f"TensorFlow Version: {tf.__version__}")
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")

TensorFlow Version: 2.18.0
Num GPUs Available: 1


### 1.3 Configuración de Constantes y Rutas del Proyecto

In [4]:
# --- Rutas del Dataset ---
# DRIVE_PROJECT = pathlib.Path('path/a/los/datos/') # Varía para cada uno, el mío es:
DRIVE_PROJECT = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/')
BASE_PATH = DRIVE_PROJECT
PATH_CURATED = BASE_PATH / 'train_curated'
PATH_TEST = BASE_PATH / 'test'

PATH_NOISY = BASE_PATH / 'train_noisy'
DF_NOISY_PATH = BASE_PATH / 'train_noisy.csv'
DF_NOISY_MULTIHOT_PATH = BASE_PATH / 'train_noisy_multihot.csv'
TFRECORD_NOISY_DIR = BASE_PATH / 'tfrecords_noisy'

# --- Archivos CSV ---
DF_CURATED_PATH = BASE_PATH / 'train_curated.csv'

DF_SUBMISSION_PATH = BASE_PATH / 'sample_submission_v24.csv'
VOCABULARY_PATH = BASE_PATH / ('vocabulary.csv')

# --- Parámetros de Audio y Espectrogramas ---
# Remuestreamos a 16kHz para aligerar el procesamiento.
# Los clips originales están a 44.1kHz
SAMPLE_RATE = 16000
WINDOW_LENGTH_SECONDS = 1
WINDOW_STEP_SECONDS = 0.5

# Parámetros para Mel-Spectrogram
N_FFT = 1024       # Longitud de la ventana FFT
HOP_LENGTH = 256   # Salto entre ventanas
N_MELS = 128       # Número de bandas Mel (común)
FMIN = 20          # Frecuencia mínima
FMAX = SAMPLE_RATE / 2 # Frecuencia máxima

BATCH_SIZE = 64

---
## 2. Análisis Exploratorio de Datos

### 2.1 Carga de Metadatos y Vocabulario

In [ ]:
# Leemos los archivos CSV que contienen los nombres de archivo y las etiquetas
#df_curated = pd.read_csv(DF_CURATED_PATH)
df_noisy = pd.read_csv(DF_NOISY_PATH)
# df_submission = pd.read_csv(DF_SUBMISSION_PATH)

# Mapear strings a índices
df_vocab = pd.read_csv(VOCABULARY_PATH, header=None, names=['id', 'label'])
LABELS = df_vocab['label'].tolist()
NUM_CLASSES = len(LABELS)
label_string_to_label_index = {string: index for index, string in enumerate(LABELS)}
label_index_to_label_string = {index: string for index, string in enumerate(LABELS)}

print(f"Número total de clases: {NUM_CLASSES}")
print("Ejemplo de etiquetas:", LABELS[:5])

#print('Curated:')
#print(df_curated.head())
print('\nNoisy:')
print(df_noisy.head())


### 2.2 Visualización de distribución de clases

In [ ]:
"""Convierte las etiquetas de string a un formato multi-hot-encoding."""
def process_labels(df, label_to_id_map):
    df['labels_list'] = df['labels'].str.split(',')

    # Creamos la matriz de one-hot encoding
    labels_matrix = np.zeros((len(df), len(label_to_id_map)), dtype=np.float32)
    for i, row in enumerate(df['labels_list']):
        for label in row:
            if label in label_to_id_map:
                labels_matrix[i, label_to_id_map[label]] = 1.0

    return labels_matrix

# Aplicamos la función a nuestros dataframes
# Hay que usarla también en los noisy luego
y_noisy = process_labels(df_noisy, label_string_to_label_index)

print("Forma de las etiquetas del conjunto curado:", y_noisy.shape)
print(y_noisy[0])

plt.figure(figsize=(15, 20))
class_counts = pd.Series(y_noisy.sum(axis=0), index=LABELS).sort_values(ascending=True)
plt.barh(class_counts.index, class_counts.values)
plt.title('Distribución de Clases en el Conjunto Noisy')
plt.xlabel('Número de Ocurrencias')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

### 2.4 Visualización de un Ejemplo: Audio y Espectrograma

Convert Audio Tensor to Mel Spectogram

In [ ]:
def plot_spectogram(log_mel_spectrogram, frame_step, sample_rate):
    log_mel_spectrogram_np = log_mel_spectrogram.numpy()
    # Calculate time axis for proper x-axis scaling
    total_frames = log_mel_spectrogram_np.shape[0]
    time_duration = (total_frames * frame_step) / sample_rate

    # Plot the mel spectrogram
    plt.figure(figsize=(10, 4))
    plt.imshow(log_mel_spectrogram_np.T, aspect='auto', origin='lower',
                extent=[0, time_duration, FMIN, FMAX])
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()
    plt.show()

Convert Audio Tensor to MFCSS

In [ ]:
def convert_audio_tensor_to_mfccs(audio_tensor, sample_rate, frame_length=N_FFT, frame_step=HOP_LENGTH, window_fn=tf.signal.hann_window, plot=True):
    spectogram = convert_audio_tensor_to_mel_spectogram(audio_tensor, sample_rate, frame_length=frame_length, frame_step=frame_step, window_fn=window_fn, plot=plot)
    mfcss = tf.signal.mfccs_from_log_mel_spectrograms(spectogram)[..., :13]
    if plot:
        mfcss_np = mfcss.numpy()
        # Calculate time axis for proper x-axis scaling
        total_frames = mfcss_np.shape[0]
        time_duration = (total_frames * frame_step) / sample_rate

        plt.figure(figsize=(10, 4))
        plt.imshow(mfcss_np.T, aspect='auto', origin='lower',
                   extent=[0, time_duration, 0, mfcss_np.shape[1]])
        plt.colorbar(format='%+2.0f dB')
        plt.title('MFCCs')
        plt.xlabel('Time (s)')
        plt.ylabel('MFCC')
        plt.tight_layout()
        plt.show()

    return mfcss


In [ ]:
def plot_wavelength(audio, sample_rate):
    # Visualizar
    plt.figure(figsize=(12, 8))
    # Waveform
    time_axis = np.arange(len(audio)) / sample_rate
    plt.plot(time_axis, audio)
    plt.title("Forma de Onda (Waveform)")
    plt.ylabel("Amplitud")
    plt.xlabel("Tiempo (s)")
    plt.xlim(0,15)
    plt.show()

Load Sample, convert to MFCSS and display

In [ ]:
def load_sample_and_convert_to_mfccs(filepath='sample.wav', sample_rate=22050):
    audio, sample_rate = load_audio_with_librosa(filepath, sample_rate)
    audio_tensor = tf.convert_to_tensor(audio, dtype=tf.float32)

    plot_wavelength(audio, sample_rate)
    mfcss = convert_audio_tensor_to_mfccs(audio_tensor, sample_rate)
    display(Audio(audio, rate=sample_rate))


Plot Audio and Spectogram

In [ ]:
def plot_audio_and_spectrogram(filepath, sr=SAMPLE_RATE):
    # Cargar audio
    audio_binary = tf.io.read_file(str(filepath))
    waveform, original_sr = tf.audio.decode_wav(audio_binary)
    waveform = tf.squeeze(waveform, axis=-1)

    # Remuestrear (tf.signal no tiene resample, se puede usar librosa o asumir pre-procesado)
    # Por simplicidad acá, asumimos que los archivos ya están a TARGET_SR o que esta función se adapta.
    # Después se podría usar la librería librosa para el remuestreo.
    print(f"Frecuencia de muestreo original: {original_sr.numpy()} Hz")

    # Calcular espectrograma Mel
    stfts = tf.signal.stft(waveform, frame_length=N_FFT, frame_step=HOP_LENGTH, fft_length=N_FFT)
    spectrograms = tf.abs(stfts)

    mel_spectrograms = tf.tensordot(
        spectrograms,
        tf.signal.linear_to_mel_weight_matrix(N_MELS, spectrograms.shape[-1], sr, FMIN, FMAX),
        1)

    log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

    # Visualizar
    fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

    # Waveform
    time_axis = np.arange(len(waveform)) / original_sr.numpy()
    axes[0].plot(time_axis, waveform.numpy())
    axes[0].set_title("Forma de Onda (Waveform)")
    axes[0].set_ylabel("Amplitud")

    # Spectrogram
    img = axes[1].imshow(tf.transpose(log_mel_spectrograms).numpy(),
                         aspect='auto', origin='lower',
                         extent=[time_axis.min(), time_axis.max(), FMIN, FMAX])
    axes[1].set_title("Espectrograma Mel (Log)")
    axes[1].set_xlabel("Tiempo (s)")
    axes[1].set_ylabel("Frecuencia (Hz)")
    fig.colorbar(img, ax=axes[1], format='%+2.0f dB')

    plt.show()

    # Reproducir audio
    display(Audio(waveform.numpy(), rate=original_sr.numpy()))


Plot one sample

In [ ]:
# Tomamos un ejemplo del conjunto curado
sample_row = df_noisy.iloc[10]
sample_path = PATH_NOISY / sample_row['fname']
print(f"Archivo: {sample_row['fname']}")
print(f"Etiquetas: {sample_row['labels']}")
#plot_audio_and_spectrogram(sample_path)
load_sample_and_convert_to_mfccs(sample_path)

---
## 3. Preprocesamiento y Pipeline de Datos

### 3.0 Conversión de labels a multihot

Parse Label string and Encode to multihot

In [9]:
def encode_label_string_as_multihot(labels_string, num_classes=NUM_CLASSES, label_string_to_label_index=label_string_to_label_index):
    assert label_string_to_label_index is not None

    """Convert '0,2,3' to [1,0,1,1,0,...]"""
    multihot = [0.0] * num_classes

    try:
        split_labels_strings = labels_string.split(',')

        clean_labels = [label.strip() for label in split_labels_strings]

        for label_string in clean_labels:
            label_index = label_string_to_label_index[label_string]
            if 0 <= label_index < num_classes:
                multihot[label_index] = 1.0
            else:
                print(f"Warning: Label {label_index} out of range [0, {num_classes-1}]")

        return np.array(multihot, dtype=np.float32)
    except Exception as e:
        print(f"Error parsing labels '{labels_string}': {e}")
        return np.array(multihot, dtype=np.float32)

In [10]:
def encode_labels_to_multihot(data_frame_string_labels, num_classes=NUM_CLASSES, label_string_to_label_index=label_string_to_label_index):
    data_frame_multihot_labels = data_frame_string_labels.copy()
    data_frame_multihot_labels['labels'] = data_frame_string_labels['labels'].apply(
        lambda label_string: encode_label_string_as_multihot(label_string, num_classes=num_classes, label_string_to_label_index=label_string_to_label_index))
    return data_frame_multihot_labels

In [11]:

try:
    df_train_noisy_multihot = pd.read_csv(DF_NOISY_MULTIHOT_PATH)
except FileNotFoundError:
    df_train_noisy = pd.read_csv(DF_NOISY_PATH)
    df_train_noisy_multihot = encode_labels_to_multihot(df_train_noisy)
    df_train_noisy_multihot.to_csv(DF_NOISY_MULTIHOT_PATH, index=False)


Generate

### 3.1 Funciones de Preprocesamiento

Load and Decode audio

In [12]:
def load_and_decode_audio(file_path):
    """
    Loads and decodes a WAV file usando solo TensorFlow operations.
    """
    audio_binary = tf.io.read_file(file_path)
    waveform, _ = tf.audio.decode_wav(audio_binary)
    waveform = tf.squeeze(waveform, axis=-1)
    return waveform

Split audio clip in same-length windows

In [13]:
def split_audio_in_windows(audio, window_length_seconds=WINDOW_LENGTH_SECONDS, window_step_seconds=WINDOW_STEP_SECONDS, sample_rate=SAMPLE_RATE):
    window_samples = int(window_length_seconds * sample_rate)
    hop_samples = int(window_step_seconds * sample_rate)

    windows = tf.signal.frame(
        signal=audio,
        frame_length=window_samples,
        frame_step=hop_samples,
        pad_end=True,
        pad_value=0.0
    )
    return windows

Create window's spectogram

In [14]:
def convert_audio_tensor_to_mel_spectogram(audio_tensor, sample_rate, frame_length=N_FFT, frame_step=HOP_LENGTH, window_fn=tf.signal.hann_window, plot=True):
    stft = tf.signal.stft(audio_tensor, frame_length=frame_length, frame_step=frame_step, window_fn=window_fn, fft_length=N_FFT)
    spectrogram = tf.abs(stft)

    num_spectrogram_bins = tf.shape(stft)[-1]
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(N_MELS,
                                                                        num_spectrogram_bins,
                                                                        sample_rate,
                                                                        FMIN,
                                                                        FMAX)

    mel_spectrogram = tf.tensordot(spectrogram, linear_to_mel_weight_matrix, 1)

    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)

    if plot:
        plot_spectogram(log_mel_spectrogram, frame_step, sample_rate)


    return log_mel_spectrogram

In [16]:
def convert_multihot_string_to_multihot(labels_string):
    numpy_data = np.array(labels_string, dtype=np.float32)
    return tf.convert_to_tensor(numpy_data, dtype=tf.float32)

Generate audio windows from file path

In [15]:
def generate_windows(file_path, label):
    audio = load_and_decode_audio(file_path)
    windows = split_audio_in_windows(audio)
    spectrograms = tf.map_fn(
        lambda x: convert_audio_tensor_to_mel_spectogram(x, plot=False, sample_rate=SAMPLE_RATE),
        windows,
        fn_output_signature=tf.float32
    )

    # Repeat the label for each spectrogram window
    num_windows = tf.shape(spectrograms)[0]
    labels = tf.repeat([label], repeats=num_windows, axis=0)

    return tf.data.Dataset.from_tensor_slices((spectrograms, labels))


### 3.2 Creación del Pipeline `tf.data`

In [ ]:
df_noisy_multihot = pd.read_csv(DF_NOISY_MULTIHOT_PATH)

# Fix: Parse the string representation of the list into a list of floats
# without using eval. Remove the brackets and split by space and comma.
labels_array = np.array([
    [float(item) for item in label.strip("[]").replace(",", " ").split()]
    for label in df_noisy_multihot['labels']
], dtype=np.float32)


full_paths = [str(PATH_NOISY / file_name) for file_name in df_noisy_multihot['fname']]

# (file_paths), (labels) ->  (file_path, label)
initial_dataset = tf.data.Dataset.from_tensor_slices((full_paths, labels_array))

# (file_path, label) -> (audio_clip, label) -> (window, label) -> (spectrogram, label)
processed_dataset = initial_dataset.flat_map(generate_windows)

# 4. Write the dataset to TFRecord files (Sharding)
SHARDS = 16 # Split into 16 files
output_dir = TFRECORD_NOISY_DIR
tf.io.gfile.makedirs(str(output_dir))

print("Writing TFRecord files...")
# Calculate items per shard to improve progress reporting accuracy
items_per_shard = len(df_noisy_multihot) // SHARDS
writer = None # Initialize writer outside the loop

for i, (spectrogram, label) in enumerate(processed_dataset):
    shard_index = i % SHARDS
    output_path = str(output_dir / f'shard-{shard_index:02d}-of-{SHARDS:02d}.tfrecord')

    # Use a new writer for the first item of each shard or if it's the very first item
    if i == 0 or i % items_per_shard == 0:
         if writer is not None:
             writer.close()
         print(f"Writing to shard {shard_index}...")
         writer = tf.io.TFRecordWriter(output_path)

    example = serialize_example(spectrogram, label)
    writer.write(example)

# Ensure the last writer is closed after the loop finishes
if writer is not None:
    writer.close()

print("Finished writing all TFRecord files.")

In [ ]:
def create_spectogram_dataset(folder_path, multihot_labels, batch_size=BATCH_SIZE, sample_rate=SAMPLE_RATE,
                             frame_length=N_FFT, frame_step=HOP_LENGTH, window_fn=tf.signal.hann_window, shuffle=True):

    # Fix: Parse the string representation of the list into a list of floats
    # without using eval. Remove the brackets and split by space and comma.
    labels_array = np.array([
      [float(item) for item in label.strip("[]").replace(",", " ").split()]
      for label in df_noisy_multihot['labels']
    ], dtype=np.float32)


    full_paths = [str(PATH_NOISY / file_name) for file_name in df_noisy_multihot['fname']]

    # (file_paths), (labels) ->  (file_path, label)
    initial_dataset = tf.data.Dataset.from_tensor_slices((full_paths, labels_array))

    print("Loading .wav audio files...")
    dataset = dataset.map(lambda file_path, labels: (load_audio_with_librosa_wrapper(file_path, sample_rate), labels),
                        num_parallel_calls=tf.data.AUTOTUNE)

    print("Splitting audio in windows...")
    dataset = dataset.map(lambda audio, labels: (split_audio_in_windows(audio), labels),
                        num_parallel_calls=tf.data.AUTOTUNE)

    print("Flattening...")
    dataset = dataset.flat_map(
        lambda windows, labels: tf.data.Dataset.from_tensor_slices((
            windows,
            tf.repeat([labels], tf.shape(windows)[0], axis=0)
        ))
    )

    print("Converting audio tensor to mel spectrogram...")
    dataset = dataset.map(lambda window, labels: (convert_audio_tensor_to_mel_spectogram(window,
                                                                                      sample_rate=sample_rate,
                                                                                      frame_length=frame_length,
                                                                                      frame_step=frame_step,
                                                                                      window_fn=window_fn,
                                                                                      plot=False), labels), num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.map(lambda spec, labels: (tf.ensure_shape(spec, [59, 128]), tf.ensure_shape(labels, [80])))

    print("Adding channel dimension to spectrogram...")
    dataset = dataset.map(lambda spectrogram, label: (
        tf.expand_dims(spectrogram, axis=-1),  # (59, 128) -> (59, 128, 1)
        label
    ))

    dataset = dataset.map(lambda spec, labels: (tf.ensure_shape(spec, [59, 128, 1]), tf.ensure_shape(labels, [80])))

    if shuffle:
        print("Shuffling...")
        dataset = dataset.shuffle(buffer_size=1000)

    print("Batching...")
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda specs, labels: (tf.ensure_shape(specs, [None, 59, 128, 1]), tf.ensure_shape(labels, [None, 80])))


    print("Prefetching...")
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

In [ ]:
dataset_size = tf.data.experimental.cardinality(train_noisy_dataset)
print(f"Dataset size: {dataset_size}")

# Look at one batch to verify pipeline works
for spectogram, labels in train_noisy_dataset.take(1):
    print(f"Spectogram shape: {spectogram.shape}")
    print(f"Labels shape: {labels.shape}")
    break

---
## 4. Métrica de Evaluación (`lwlrap`)

La métrica lwlrap se implementa usando el módulo lwlrap_metric.py que contiene una implementación optimizada para TensorFlow

In [ ]:
#from lwlrap_metric import LwlrapMetric

# -*- coding: utf-8 -*-
"""
Métrica Label-Weighted Label-Ranking Average Precision (lwlrap) para TensorFlow/Keras.
Contiene la implementación de la métrica para usar durante el entrenamiento y funciones
auxiliares para evaluación post-entrenamiento.
"""

import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn.metrics

#---- PARTE 1: MÉTRICA PARA TENSORFLOW/KERAS ----------------------------------#

@tf.function
def _tf_one_sample_positive_class_precisions(scores, truth):
    """
    Versión en TensorFlow para calcular las precisiones de las clases positivas
    para una única muestra.
    """
    num_classes = tf.shape(scores)[0]
    pos_class_indices = tf.where(truth > 0)[:, 0]

    if tf.size(pos_class_indices) == 0:
        return pos_class_indices, tf.constant([], dtype=tf.float32)

    retrieved_classes = tf.argsort(scores, direction='DESCENDING')

    class_rankings = tf.scatter_nd(
        tf.expand_dims(retrieved_classes, 1),
        tf.range(num_classes, dtype=tf.int32),
        [num_classes]
    )

    retrieved_class_true = tf.scatter_nd(
        tf.expand_dims(tf.gather(class_rankings, pos_class_indices), 1),
        tf.ones(tf.size(pos_class_indices), dtype=tf.bool),
        [num_classes]
    )

    retrieved_cumulative_hits = tf.cumsum(tf.cast(retrieved_class_true, tf.float32))

    rankings_pos_classes = tf.gather(class_rankings, pos_class_indices)
    precision_at_hits = (
        tf.gather(retrieved_cumulative_hits, rankings_pos_classes) /
        (tf.cast(rankings_pos_classes, tf.float32) + 1.0)
    )

    return pos_class_indices, precision_at_hits

class LwlrapMetric(keras.metrics.Metric):
    """
    Métrica lwlrap para TensorFlow/Keras.
    Funciona con operaciones de TensorFlow para ser compatible con el modo graph.
    """
    def __init__(self, num_classes, name='lwlrap', **kwargs):
        super(LwlrapMetric, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.per_class_cumulative_precision = self.add_weight(
            name='per_class_cumulative_precision',
            shape=(num_classes,),
            initializer='zeros',
            dtype=tf.float32
        )
        self.per_class_cumulative_count = self.add_weight(
            name='per_class_cumulative_count',
            shape=(num_classes,),
            initializer='zeros',
            dtype=tf.float32
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        def process_sample(sample_data):
            truth, scores = sample_data
            pos_class_indices, precision_at_hits = _tf_one_sample_positive_class_precisions(scores, truth)
            precision_update = tf.scatter_nd(
                tf.expand_dims(pos_class_indices, 1),
                precision_at_hits,
                [self.num_classes]
            )
            count_update = tf.scatter_nd(
                tf.expand_dims(pos_class_indices, 1),
                tf.ones(tf.size(pos_class_indices), dtype=tf.float32),
                [self.num_classes]
            )
            return precision_update, count_update

        precision_updates, count_updates = tf.map_fn(
            process_sample,
            (y_true, y_pred),
            fn_output_signature=(
                tf.TensorSpec([self.num_classes], dtype=tf.float32),
                tf.TensorSpec([self.num_classes], dtype=tf.float32)
            )
        )

        self.per_class_cumulative_precision.assign_add(tf.reduce_sum(precision_updates, axis=0))
        self.per_class_cumulative_count.assign_add(tf.reduce_sum(count_updates, axis=0))

    def result(self):
        per_class_lwlrap = tf.divide(
            self.per_class_cumulative_precision,
            tf.maximum(self.per_class_cumulative_count, 1.0)
        )
        total_count = tf.reduce_sum(self.per_class_cumulative_count)
        weight_per_class = tf.divide(
            self.per_class_cumulative_count,
            tf.maximum(total_count, 1.0)
        )
        overall_lwlrap = tf.reduce_sum(per_class_lwlrap * weight_per_class)
        return overall_lwlrap

    def reset_state(self):
        self.per_class_cumulative_precision.assign(tf.zeros_like(self.per_class_cumulative_precision))
        self.per_class_cumulative_count.assign(tf.zeros_like(self.per_class_cumulative_count))


# ------------------------------------------------------------------------------------
# PARTE 2: FUNCIONES DE NUMPY/SKLEARN (OPCIONAL, ÚTIL PARA ANÁLISIS POST-ENTRENAMIENTO)
# ------------------------------------------------------------------------------------

def calculate_overall_lwlrap_sklearn(truth, scores):
    """Calcula el lwlrap general usando la implementación de referencia de sklearn."""
    sample_weight = np.sum(truth > 0, axis=1)
    nonzero_weight_sample_indices = np.flatnonzero(sample_weight > 0)
    if len(nonzero_weight_sample_indices) == 0:
        return 0.0
    overall_lwlrap = sklearn.metrics.label_ranking_average_precision_score(
        truth[nonzero_weight_sample_indices, :] > 0,
        scores[nonzero_weight_sample_indices, :],
        sample_weight=sample_weight[nonzero_weight_sample_indices]
    )
    return overall_lwlrap


# Definir la métrica lwlrap para usar en Keras
lwlrap_metric = LwlrapMetric(num_classes=NUM_CLASSES)

---
## 5. Configuración de Experimentación con `Comet.ml`

### 5.1 Conexión con Comet.ml

In [ ]:
try:
    api_key = userdata.get('COMET_API_KEY')
    comet_ml.login(api_key=api_key)
    print("Conexión con Comet.ml exitosa.")
except userdata.SecretNotFoundError:
    print("⚠️ Clave API de Comet no encontrada. Guardala como 'COMET_API_KEY' en los secrets de Colab (a la izquierda).")
except Exception as e:
    print(f"Error al conectar con Comet.ml: {e}")

# Esta función ayuda a iniciar cada experimento de forma limpia.
# Revisar esto
def create_comet_experiment(name, tags, params):
    """Crea y configura un nuevo experimento en Comet."""
    experiment = comet_ml.Experiment(
        project_name="freesound-audio-tagging-2019",
        auto_metric_logging=True,
        auto_param_logging=True
    )
    experiment.set_name(name)
    experiment.add_tags(tags)
    experiment.log_parameters(params)
    return experiment

---
## 6. Experimentos

### 6.1 Experimento 1: Línea Base

**Hipótesis:** Una CNN simple, entrenada solo con los datos curados, puede aprender a diferenciar algunas de las clases más comunes.

**Configuración:**
-   **Datos:** Conjunto curado (80/20 train/val).
-   **Arquitectura:** CNN 2D simple (2 bloques Conv + Pool).
-   **Pérdida:** `BinaryCrossentropy` (adecuada para multi-label).
-   **Métricas:** `lwlrap`.

#### 6.1.1 Definición del Modelo Base

In [ ]:
def build_baseline_model(input_shape, num_classes):
    """Construye un modelo CNN 2D simple."""
    # Add a channel dimension to the input shape for Conv2D
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_shape)),

        # Completar la arquitectura. Ver `simple_audio.py`o pensar otra
        # Un buen punto de partida seria:
        # Bloque 1: Conv2D(32, (3,3), activation='relu') -> MaxPool2D()
        # Bloque 2: Conv2D(64, (3,3), activation='relu') -> MaxPool2D()
        # Aplanado: GlobalAveragePooling2D() o Flatten()
        # Clasificador: Dense(128, activation='relu') -> Dropout(0.5) -> Dense(num_classes, activation='sigmoid')

        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.GlobalAveragePooling2D(),

        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(num_classes, activation='sigmoid') # Sigmoid para multi-label
    ])
    return model

# Construimos y compilamos el modelo
INPUT_SHAPE = [59, 128, 1]
baseline_model = build_baseline_model(INPUT_SHAPE, NUM_CLASSES)
baseline_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[lwlrap_metric]
)
baseline_model.summary()

#### 6.1.2 Entrenamiento del Modelo Base

In [ ]:
# Profile your data pipeline
import time

print("=== DATA PIPELINE PROFILING ===")
dataset_iter = iter(train_noisy_dataset)

# Time data loading
start_time = time.time()
for i in range(10):
    batch = next(dataset_iter)
    if i == 0:
        print(f"Batch shape: {batch[0].shape}")
    print(f"Batch {i+1}: {(time.time() - start_time)*1000:.1f}ms")
    start_time = time.time()

In [ ]:
# Configuración del experimento en Comet
exp_baseline_params = {
    'model_type': 'baseline_cnn',
    'dataset': 'training_only',
    'epochs': 20,
    'batch_size': 64
}
exp_baseline = create_comet_experiment(
    name="Baseline CNN",
    tags=["baseline", "cnn", "training-only"],
    params=exp_baseline_params
)

# Callbacks
#early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_lwlrap', patience=5, mode='max', restore_best_weights=True)
comet_callback = exp_baseline.get_callback('keras')

# Entrenamiento
history_baseline = baseline_model.fit(
    train_noisy_dataset,
    epochs=exp_baseline_params['epochs'],
    callbacks=[comet_callback]
)

# Fin
exp_baseline.end()

### 6.2 Experimento 2: Mejoras sobre la Línea Base

**Hipótesis:** Añadir regularización (Batch Norm, Dropout más robusto) y Data Augmentation mejorará la generalización y el score `lwlrap`.

**Configuración:**
-   **Datos:** Mismos datos, pero con aumento de datos en el pipeline.
-   **Arquitectura:** CNN mejorada (más profunda, con `BatchNormalization`).
-   **Técnicas:** `SpecAugment` (enmascaramiento en frecuencia y tiempo).

#### 6.2.1 Data Augmentation (SpecAugment)

In [ ]:
# Completar: Implementar SpecAugment.
# Esqueleto:
import tensorflow_addons as tfa

@tf.function
def augment_spectrogram(spectrogram, label):
    """Aplica SpecAugment a un espectrograma."""
    # Enmascaramiento en el tiempo
    spectrogram = tfa.image.random_cutout(
        images=tf.expand_dims(spectrogram, 0), # Requiere un batch
        mask_size=(10, 20), # (alto, ancho) del recorte -> (freq, tiempo)
    )[0] # Deshacer el batch

    # Enmascaramiento en frecuencia
    spectrogram = tfa.image.random_cutout(
        images=tf.expand_dims(spectrogram, 0),
        mask_size=(20, 10),
    )[0]

    return spectrogram, label

# Creamos un nuevo pipeline de entrenamiento con augmentation
train_ds_aug = train_ds.map(augment_spectrogram, num_parallel_calls=tf.data.AUTOTUNE)

#### 6.2.2 Definición del Modelo Mejorado

In [ ]:
def build_improved_model(input_shape, num_classes):
    """Construye una CNN más robusta con BatchNormalization."""
    # Mejorar el modelo base. Probar el patrón Conv -> BatchNorm -> ReLU -> Pool.
    # Se puede añadir más bloques para hacerlo más profundo.

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        # Bloque 1
        tf.keras.layers.Conv2D(32, (3,3), padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPooling2D((2,2)),

        # Bloque 2
        tf.keras.layers.Conv2D(64, (3,3), padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPooling2D((2,2)),

        # Bloque 3
        tf.keras.layers.Conv2D(128, (3,3), padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),

        tf.keras.layers.GlobalAveragePooling2D(),

        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='sigmoid')
    ])

    return model

# Compilamos
improved_model = build_improved_model(INPUT_SHAPE, NUM_CLASSES)
improved_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[lwlrap_tf]
)
improved_model.summary()

#### 6.2.3 Entrenamiento del Modelo Mejorado

In [ ]:
# Configuración del experimento
exp_improved_params = {
    'model_type': 'improved_cnn',
    'dataset': 'curated_only',
    'augmentation': 'specaugment',
    'epochs': 40, # Más épocas por la regularización
    'batch_size': 32
}
exp_improved = create_comet_experiment(
    name="Improved CNN + SpecAugment",
    tags=["improved", "cnn", "specaugment"],
    params=exp_improved_params
)

# Callbacks
early_stopping_imp = tf.keras.callbacks.EarlyStopping(monitor='val_lwlrap_tf', patience=7, mode='max', restore_best_weights=True)
comet_callback_imp = exp_improved.get_callback('keras')

# Entrenamiento
history_improved = improved_model.fit(
    train_ds_aug, # Usamos el dataset con augmentation
    epochs=exp_improved_params['epochs'],
    validation_data=val_ds,
    callbacks=[early_stopping_imp, comet_callback_imp]
)
exp_improved.end()

### 6.3 Experimento 3: Enfoque Avanzado

**Hipótesis:** Aprovechar un modelo pre-entrenado (Transfer Learning) o los datos ruidosos puede mejorar significativamente el rendimiento, ya que nuestro conjunto curado es pequeño.

**Opción A: Transferencia de Aprendizaje (Transfer Learning)**
Podemos usar un modelo pre-entrenado en ImageNet, como `EfficientNetB0`.

#### 6.3.1 Definición del Modelo con Transfer Learning

In [ ]:
def build_transfer_model(input_shape, num_classes):
    """Construye un modelo usando EfficientNetB0 pre-entrenado."""
    # Los modelos de ImageNet esperan 3 canales. Duplicamos nuestro canal único.
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Concatenate(axis=-1)([inputs, inputs, inputs])

    # Cargamos el modelo base sin el clasificador
    base_model = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_tensor=x
    )

    # Congelamos el modelo base para no destruir los pesos pre-entrenados
    base_model.trainable = False

    # Añadimos nuestro propio clasificador
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model, base_model

# El entrenamiento con Transfer Learning se suele hacer en dos fases:
# 1. Entrenar solo el clasificador nuevo con el modelo base congelado.
# 2. Descongelar algunas capas del modelo base y re-entrenar (fine-tuning) con una tasa de aprendizaje muy baja.

# --- FASE 1: ENTRENAMIENTO DEL CLASIFICADOR ---
transfer_model, base_model = build_transfer_model(INPUT_SHAPE, NUM_CLASSES)
transfer_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[lwlrap_tf]
)
transfer_model.summary()

# Entrenamiento fase 1... (similar a los anteriores)

# --- FASE 2: FINE-TUNING (Opcional pero recomendado) ---
# base_model.trainable = True
# # Descongelar solo las últimas capas es una buena práctica
# for layer in base_model.layers[:-20]:
#     layer.trainable = False
#
# transfer_model.compile(
#     optimizer=tf.keras.optimizers.Adam(1e-5), # Tasa de aprendizaje muy baja
#     loss=tf.keras.losses.BinaryCrossentropy(),
#     metrics=[lwlrap_tf]
# )
#
# Entrenamiento fase 2...

**Opción B: Uso de Datos Ruidosos (Pre-training / Fine-tuning)**
Esta es una estrategia poderosa.
1.  **Pre-entrenamiento:** Entrenar un modelo en el gran conjunto de datos `ruidoso`. El modelo aprenderá características de audio generales, a pesar de las etiquetas imperfectas.
2.  **Fine-tuning:** Tomar el modelo pre-entrenado y continuar su entrenamiento en el conjunto `curado`, que es más pequeño pero de alta calidad.

In [ ]:
# 1. Crear un `tf.data.Dataset` para el conjunto ruidoso (`df_noisy`).
# 2. Entrenar uno de los modelos (ej. el `improved_model`) en este dataset ruidoso por varias épocas.
# 3. Guardar los pesos de este modelo.
# 4. Cargar esos pesos en un nuevo modelo y entrenarlo en el dataset curado (`train_ds`).
# ¡No olvidar registrar ambos pasos en Comet!

## 7. Generación de la Submission para Kaggle

Una vez que tengamos **el mejor modelo**, lo usamos para predecir las etiquetas en el conjunto de prueba.

### 7.1 Creación del Pipeline de Test

In [ ]:
def create_test_dataset(df, path_prefix):
    filepaths = [str(path_prefix / fname) for fname in df['fname']]
    dataset = tf.data.Dataset.from_tensor_slices(filepaths)

    # El preprocesamiento debe ser idéntico al de validación
    def preprocess_test_file(filepath):
        waveform = load_audio(filepath)
        waveform = pad_or_truncate(waveform)
        spectrogram = to_mel_spectrogram(spectrogram)
        return spectrogram

    dataset = dataset.map(preprocess_test_file, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(64) # Usar un batch size grande para acelerar la predicción
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

test_ds = create_test_dataset(df_submission, PATH_TEST)

# Usamos nuestro mejor modelo para predecir
# A A A A A A A   ¡¡Reemplazar `improved_model` por el mejor!!
best_model = improved_model
predictions = best_model.predict(test_ds)

### 7.2 Formateo y Guardado del Archivo de Submission

In [ ]:
# Creamos el dataframe de submission con las predicciones
submission_df = pd.DataFrame(predictions, columns=LABELS)
submission_df['fname'] = df_submission['fname']

# Reordenamos las columnas para que coincida con el formato de `sample_submission.csv`
submission_df = submission_df[df_submission.columns]

# Guardamos el archivo
submission_df.to_csv('submission.csv', index=False)
print("Archivo submission.csv creado.")
submission_df.head()

## 8. Comparación y Análisis Final

Acá es donde reflexionamos sobre nuestros resultados :)

## 8.1 Tabla Comparativa de Resultados

In [ ]:
# Ir al dashboard de Comet.ml, buscar los scores de `val_lwlrap_tf` de cada experimento
# y completar esta tabla:

results_data = {
    'Experimento': ['1. Línea Base', '2. Mejorado + Augment', '3. Avanzado (TL/Ruidoso)'],
    'Mejor val_lwlrap': [0.0, 0.0, 0.0], # <-- COMPLETAR ACÁ
    'Notas': [
        'CNN simple, solo datos curados.',
        'CNN más profunda con BN y SpecAugment.',
        'Resultados del mejor enfoque avanzado.'
    ]
}
results_df = pd.DataFrame(results_data)
display(results_df)

¿Qué conclusiones podemos sacar?
- ¿Funcionó la data augmentation?
- ¿Cuál fue el impacto de una arquitectura más profunda?
- Si probamos un enfoque avanzado, ¿cuál fue el beneficio?